In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV,ElasticNet,ElasticNetCV,LinearRegression

In [2]:
df = pd.read_csv(r"C:\Users\krunal sheth\Downloads\WEC_Perth_49.csv")
df.head()

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5,...,Power42,Power43,Power44,Power45,Power46,Power47,Power48,Power49,qW,Total_Power
0,600.0,0.0,546.16,37.50,489.79,74.88,432.47,112.05,650.0,0.0,...,88867.92,98844.30,101283.59,98934.63,101624.58,100915.03,99625.68,96704.34,0.87,4102461.43
1,593.0,12.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88896.55,98759.79,101346.07,98873.59,101629.01,100934.53,99606.13,96718.39,0.87,4103361.41
2,593.0,12.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88919.83,98746.68,101346.15,98875.57,101618.32,100941.00,99611.35,96719.14,0.87,4103680.44
3,593.0,12.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88855.14,98760.96,101338.59,98971.58,101632.28,100943.59,99589.25,96735.04,0.87,4105661.06
4,200.0,0.0,146.17,37.53,89.76,74.93,32.40,112.18,400.0,0.0,...,88005.30,98630.24,100432.73,98803.01,101064.48,100948.38,99028.87,96286.71,0.79,3752648.77


In [3]:
df.shape

(36043, 149)

In [4]:
df.isnull().sum()

X1             0
Y1             0
X2             0
Y2             0
X3             0
              ..
Power47        0
Power48        0
Power49        0
qW             0
Total_Power    0
Length: 149, dtype: int64

In [5]:
df.duplicated().sum()

25107

In [6]:
def replace_outliers_with_nan(df):
    df_num = df
    for col in (df):
        Q1 = df_num[col].quantile(0.25)
        Q3 = df_num[col].quantile(0.75)
        IQR = Q3 - Q1
        upper_limit = Q3 + 1.5 * IQR
        lower_limit = Q1 - 1.5 * IQR
        df_num[col] = df_num[col].mask((df_num[col] < lower_limit) | (df_num[col] > upper_limit), np.nan)
    return df_num
    
df_with_nans = replace_outliers_with_nan(df)
    

In [7]:
df.isnull().sum()

X1                0
Y1             7968
X2                0
Y2             4814
X3                0
               ... 
Power47        3753
Power48        3474
Power49        7160
qW                6
Total_Power      24
Length: 149, dtype: int64

In [8]:
df = df.fillna(df.median())

In [9]:
df.shape

(36043, 149)

In [10]:
df.isnull().sum()

X1             0
Y1             0
X2             0
Y2             0
X3             0
              ..
Power47        0
Power48        0
Power49        0
qW             0
Total_Power    0
Length: 149, dtype: int64

In [11]:
df.head()

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5,...,Power42,Power43,Power44,Power45,Power46,Power47,Power48,Power49,qW,Total_Power
0,600.0,0.0,546.16,37.50,489.79,74.88,432.47,112.05,650.0,0.0,...,88867.92,98844.30,101283.59,98934.63,101624.58,100915.03,99625.68,96704.34,0.87,4102461.43
1,593.0,0.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88896.55,98759.79,101346.07,98873.59,101629.01,100934.53,99606.13,96718.39,0.87,4103361.41
2,593.0,0.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88919.83,98746.68,101346.15,98875.57,101618.32,100941.00,99611.35,96719.14,0.87,4103680.44
3,593.0,0.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,...,88855.14,98760.96,101338.59,98971.58,101632.28,100943.59,99589.25,96735.04,0.87,4105661.06
4,200.0,0.0,146.17,37.53,89.76,74.93,32.40,112.18,400.0,0.0,...,88005.30,98630.24,100432.73,98803.01,101064.48,100948.38,99028.87,96286.71,0.79,3752648.77


In [12]:
df.drop(['Y1'],axis = 1, inplace = True )

In [13]:
y = df[['Total_Power']]
y.head(3)

,Total_Power
0,4102461.43
1,4103361.41
2,4103680.44


In [14]:
x = df.drop(['Total_Power'],axis = 1)
x.head(5)

,X1,X2,Y2,X3,Y3,X4,Y4,X5,Y5,X6,...,Power41,Power42,Power43,Power44,Power45,Power46,Power47,Power48,Power49,qW
0,600.0,546.16,37.50,489.79,74.88,432.47,112.05,650.0,0.0,700.00,...,77097.57,88867.92,98844.30,101283.59,98934.63,101624.58,100915.03,99625.68,96704.34,0.87
1,593.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,700.00,...,77044.77,88896.55,98759.79,101346.07,98873.59,101629.01,100934.53,99606.13,96718.39,0.87
2,593.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,697.00,...,77038.66,88919.83,98746.68,101346.15,98875.57,101618.32,100941.00,99611.35,96719.14,0.87
3,593.0,546.16,37.50,489.79,74.88,432.47,112.05,644.0,8.0,697.00,...,77057.86,88855.14,98760.96,101338.59,98971.58,101632.28,100943.59,99589.25,96735.04,0.87
4,200.0,146.17,37.53,89.76,74.93,32.40,112.18,400.0,0.0,346.17,...,76869.43,88005.30,98630.24,100432.73,98803.01,101064.48,100948.38,99028.87,96286.71,0.79


In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
# standard scalerisation

scaler = StandardScaler()

In [17]:
arr = scaler.fit_transform(x)
arr

array([[ 0.7580307 ,  0.45092571,  0.52648571, ...,  1.08450089,
         0.28327598,  1.38894395],
       [ 0.73529656,  0.45092571,  0.52648571, ...,  1.06698969,
         0.31713413,  1.38894395],
       [ 0.73529656,  0.45092571,  0.52648571, ...,  1.07166531,
         0.31894151,  1.38894395],
       ...,
       [-0.54106296, -0.09900771,  0.52375421, ...,  0.38138352,
        -0.16806169,  1.00455172],
       [ 0.43325728, -0.2871349 ,  0.52375421, ...,  0.38138352,
         2.01717509,  1.00455172],
       [-0.86583638, -0.09900771,  0.52375421, ...,  2.68511447,
        -0.16806169,  1.00455172]])

In [18]:
# convert array in to data structure
df1 = pd.DataFrame(arr)
df1.head(5)

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,0.758031,0.450926,0.526486,0.046221,0.210031,-0.231309,0.851058,-0.466743,-0.738045,-0.066863,...,-1.290920,-0.687447,0.171641,1.303862,0.089091,0.756848,0.670838,1.084501,0.283276,1.388944
1,0.735297,0.450926,0.526486,0.046221,0.210031,-0.231309,0.851058,-0.501235,-0.513315,-0.066863,...,-1.296534,-0.683453,0.151691,1.347452,0.011622,0.759095,0.679851,1.066990,0.317134,1.388944
2,0.735297,0.450926,0.526486,0.046221,0.210031,-0.231309,0.851058,-0.501235,-0.513315,-0.080079,...,-1.297183,-0.680206,0.148596,1.347508,0.014135,0.753672,0.682841,1.071665,0.318942,1.388944
3,0.735297,0.450926,0.526486,0.046221,0.210031,-0.231309,0.851058,-0.501235,-0.513315,-0.080079,...,-1.295142,-0.689230,0.151967,1.342233,0.135986,0.760754,0.684038,1.051870,0.357258,1.388944
4,-0.541063,-1.054054,0.528125,-1.433627,0.213798,-1.662034,0.853819,-1.903892,-0.738045,-1.625667,...,-1.315175,-0.807773,0.121109,0.710255,-0.077954,0.472692,0.686252,0.549930,-0.723143,-1.686194


In [19]:
df1.describe()

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
count,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,...,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04,3.604300e+04
mean,-3.154200e-17,-1.640184e-16,4.415880e-17,-8.831761e-17,-3.154200e-16,-3.154200e-16,3.532704e-16,-1.608642e-16,2.523360e-17,-5.109804e-16,...,3.185742e-16,-2.056539e-15,1.898829e-15,3.728265e-15,-1.611165e-14,-1.185979e-15,1.236446e-15,-6.914007e-15,2.914481e-15,-1.066120e-15
std,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,...,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00
min,-1.190610e+00,-1.604025e+00,-1.522138e+00,-1.765680e+00,-4.338363e+00,-1.777903e+00,-1.528966e+00,-2.823380e+00,-7.380451e-01,-2.930449e+00,...,-3.374374e+00,-3.460567e+00,-2.933886e+00,-5.167837e+00,-4.502864e+00,-4.114417e+00,-3.701116e+00,-4.315981e+00,-3.127674e+00,-3.608155e+00
25%,-9.770063e-01,-8.515165e-01,-1.522138e+00,-6.930556e-01,2.040033e-01,-7.050469e-01,-4.669292e-01,-5.308111e-01,-7.380451e-01,-7.276903e-01,...,-1.135767e+00,-7.814188e-01,-4.519068e-02,-2.811629e-01,-1.018525e-01,-8.219901e-01,-5.843702e-01,-5.454384e-01,-4.609046e-01,-9.174094e-01
50%,-3.786763e-01,-3.018464e-01,5.237542e-01,-2.859429e-01,2.115375e-01,1.019039e-02,5.951073e-01,1.081168e-01,-7.380451e-01,1.534132e-01,...,1.790806e-01,-1.861167e-03,1.654374e-01,1.664461e-01,9.170578e-02,5.069772e-01,5.410068e-01,3.813835e-01,-1.680617e-01,-1.486250e-01
75%,7.580307e-01,1.202757e+00,5.286709e-01,7.861267e-01,2.160580e-01,4.849292e-01,8.531822e-01,6.829765e-01,6.665183e-01,8.142408e-01,...,9.548403e-01,1.004503e+00,5.914196e-01,5.786980e-01,2.949096e-01,6.281835e-01,6.894738e-01,5.818398e-01,2.051733e-01,1.004552e+00
max,2.057124e+00,2.158519e+00,3.494532e+00,1.933663e+00,2.785218e+00,1.798284e+00,2.732774e+00,1.545266e+00,2.399188e+00,1.254793e+00,...,1.575878e+00,2.392201e+00,2.663591e+00,2.965225e+00,3.308967e+00,2.343153e+00,1.584639e+00,3.457963e+00,3.665453e+00,1.773336e+00


In [20]:
y1 = scaler.fit_transform(y)
y1

array([[1.34291118],
       [1.35028424],
       [1.35289788],
       ...,
       [0.89717665],
       [1.01565636],
       [0.97493445]])

In [21]:
# convert array in to data structure
df2 = pd.DataFrame(y1)
df2.head(5)

,0
0,1.342911
1,1.350284
2,1.352898
3,1.369124
4,-1.522920


In [22]:
df2.describe()

,0
count,3.604300e+04
mean,3.267751e-15
std,1.000014e+00
min,-3.395695e+00
25%,-7.456756e-01
50%,-5.296059e-02
75%,1.024726e+00
max,1.958965e+00


In [23]:
# divde data in to training and testing

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(arr, y1, test_size=0.20, random_state=85)

In [25]:
# build linear Regression model

In [26]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [27]:
lr.fit(x_train,y_train)

LinearRegression()

# save model

In [28]:
pickle.dump(lr,open('large_scale_wave_energy_Farm_lr_model.pickel','wb'))

# prediction

In [29]:
lr.predict(df1)

array([[1.3559411 ],
       [1.35421504],
       [1.35318954],
       ...,
       [1.2746848 ],
       [1.11905822],
       [1.01096189]])

# use saved pickle model

In [30]:
model = pickle.load((open('large_scale_wave_energy_Farm_lr_model.pickel','rb')))
model

LinearRegression()

In [31]:
base_preds = model.predict(df1)
base_preds

array([[1.3559411 ],
       [1.35421504],
       [1.35318954],
       ...,
       [1.2746848 ],
       [1.11905822],
       [1.01096189]])

In [32]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
y_pred = model.predict(x_test)
print('MAE:',mean_absolute_error(y_test,y_pred))
print('MSE:',mean_squared_error(y_test,y_pred))
print('R2 score:',r2_score(y_test,y_pred))

MAE: 0.07461628577994737
MSE: 0.016525549902703853
R2 score: 0.9832867561121612


In [33]:
lr.score(x_test,y_test)

0.9832867561121612

In [34]:
lr.coef_

array([[ 8.51356145e-03,  1.78285632e-02,  8.91019836e-03,
         2.36126505e-03,  1.11348541e-02,  1.42343078e-02,
         2.79964085e-02, -3.53915004e-03,  8.30533946e-04,
         8.23524788e-03, -7.07980552e-03, -1.18239353e-02,
        -9.55101841e-03,  2.20125263e-02,  5.50294519e-04,
         2.53568750e-02,  7.68194614e-04,  1.22291340e-02,
        -1.53901479e-02, -7.15272450e-03,  9.29204532e-03,
        -3.89872603e-04, -4.63078377e-03,  3.85350832e-04,
        -6.04758074e-03, -7.02364879e-03, -3.45649525e-04,
         9.11158985e-03,  1.75379647e-02, -1.05040424e-04,
         1.23064044e-02, -3.04540489e-03, -3.24106112e-03,
         1.07751191e-03, -3.28398445e-03,  9.70902166e-03,
        -6.76428767e-03, -1.01529708e-02,  1.12291008e-02,
        -3.58792922e-04, -2.67055905e-03, -4.91173392e-03,
        -5.86934898e-03, -3.63237233e-03,  8.10940559e-03,
        -7.28165697e-03, -5.65740569e-03, -7.52217855e-04,
         3.80648318e-03, -1.28638254e-02,  5.15858541e-0

In [35]:
lr.intercept_

array([-0.00046854])

# Adjusted R2

In [36]:
# created adjusted R2 function
def adj_r2(x,y):
    r2 = lr.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [37]:
adj_r2(x_test,y_test)

0.9829388100915534

# variation inflation factor

In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [39]:
arr.shape

(36043, 147)

In [40]:
vif_df = pd.DataFrame()

In [41]:
vif_df['vif'] = [variance_inflation_factor(arr,i) for i in range(arr.shape[1])]
vif_df

,vif
0,9.019485
1,7.291278
2,4.012107
3,5.401420
4,2.358124
...,...
142,2.827185
143,2.924645
144,2.417964
145,1.899598


In [42]:
vif_df['feature']= x.columns
vif_df

,vif,feature
0,9.019485,X1
1,7.291278,X2
2,4.012107,Y2
3,5.401420,X3
4,2.358124,Y3
...,...,...
142,2.827185,Power46
143,2.924645,Power47
144,2.417964,Power48
145,1.899598,Power49


# Regularization

In [43]:
# Linear Regression with Lasso
lassocv = LassoCV(cv=10, max_iter = 200000)
lassocv.fit(x_train, y_train)

C:\Users\krunal sheth\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(cv=10, max_iter=200000)

In [44]:
lassocv.alpha_

0.000992892629149291

In [45]:
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(x_train, y_train)

Lasso(alpha=0.000992892629149291)

In [46]:
y_pred = lasso.predict(x_test)
print('MSE:',mean_squared_error(y_test,y_pred))

MSE: 0.01688782032686637


In [47]:
# Linear Regression with Ridge
ridgecv = RidgeCV(alphas = np.random.uniform(0,10,50),cv=10)
ridgecv.fit(x_train, y_train)

RidgeCV(alphas=array([5.31107644e-03, 7.26032496e+00, 9.28927066e+00, 8.74949884e+00,
       2.09261865e+00, 4.57293500e+00, 7.61001346e+00, 4.44284401e+00,
       5.88043343e+00, 2.65506003e+00, 4.00639449e+00, 3.88156967e-01,
       6.81054281e+00, 4.33813676e+00, 1.69339718e+00, 3.68969949e-01,
       6.95718596e+00, 6.31331479e-01, 2.62874040e+00, 9.87702792e+00,
       1.32831429e-02, 1.10294773e+0...
       7.14469695e+00, 5.06046608e+00, 4.11611386e+00, 4.01726301e+00,
       8.36778163e+00, 1.22438449e+00, 2.73387443e+00, 6.54757833e+00,
       3.66283263e+00, 8.86886672e-01, 2.27468192e+00, 3.66221588e+00,
       6.70062722e+00, 1.59624401e+00, 4.96764534e+00, 9.35925805e+00,
       4.90294460e+00, 7.04380806e+00, 8.29303248e+00, 2.32673975e+00,
       4.37352695e+00, 2.67887529e+00, 3.17155616e+00, 3.45945516e+00,
       5.73993131e+00, 9.70521275e+00]),
        cv=10)

In [48]:
ridgecv.alpha_

0.013283142902457135

In [49]:
ridge_lr = Ridge(alpha = ridgecv.alpha_)
ridge_lr.fit(x_train, y_train)

Ridge(alpha=0.013283142902457135)

In [50]:
y_pred = ridge_lr.predict(x_test)
print('MSE:',mean_squared_error(y_test,y_pred))

MSE: 0.016525554899420817
